In [24]:
!pip3 install gql


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\HONOR\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
!pip3 install tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\HONOR\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
!pip3 install requests-toolbelt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\HONOR\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
import requests
import warnings
import math

import pandas as pd
import datetime as dt

import plotly.express as px

from tqdm import tqdm
from gql import gql, Client # to use GraphQL
from gql.transport.requests import RequestsHTTPTransport

%matplotlib inline

warnings.filterwarnings('ignore')

In [28]:
!pip3 freeze | egrep -w "pandas|numpy|matplotlib|plotly|gql|tqdm|requests"

"egrep" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


## Liquidity provision in Uniswap V3

USDC - WETH pool: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640

In [55]:
data_pool = []

# query for the USDC-WETH pool
query = gql(
    """
    {
        poolDayDatas(
            first: 1000
            orderBy: date
            orderDirection: desc
            where: {pool: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"}
        ) {
            date
            token0Price
            token1Price
            feesUSD
            liquidity
        }
    }
    """
)   


result = client.execute(query, variable_values=vars)

data = []
for i in result['poolDayDatas']:
    data.append([
        i['date'],
        i['token0Price'],
        i['token1Price'],
        i['feesUSD'],
        i['liquidity']
    ])


df = pd.DataFrame(data)
df.columns = ['date', 'token0Price', 'token1Price', 'feesUSD', 'liquidity']

df['token0Price'] = pd.to_numeric(df['token0Price'])
df['token1Price'] = pd.to_numeric(df['token1Price'])
df['feesUSD'] = pd.to_numeric(df['feesUSD'])
df['USDC/WETH'] = pd.to_numeric(df['token0Price']/df['token1Price'])
df['datetime'] = pd.to_datetime(df['date']*1000000000)

df

,date,token0Price,token1Price,feesUSD,liquidity,USDC/WETH,datetime
0,1698969600,1806.594875,0.000554,60393.827482,22016395606536397087,3.263785e+06,2023-11-03
1,1698883200,1800.035082,0.000556,137531.465707,21721162802396480979,3.240126e+06,2023-11-02
2,1698796800,1846.249690,0.000542,145129.978936,20678237003634529799,3.408638e+06,2023-11-01
3,1698710400,1816.150815,0.000551,99073.055545,21509722104600588513,3.298404e+06,2023-10-31
4,1698624000,1809.635935,0.000553,115315.482355,24518778025093597202,3.274782e+06,2023-10-30
...,...,...,...,...,...,...,...
908,1620518400,3909.453752,0.000256,3165.481894,191200334840491900,1.528383e+07,2021-05-09
909,1620432000,3884.370431,0.000257,10666.832264,87209657035771435,1.508833e+07,2021-05-08
910,1620345600,3483.701459,0.000287,12066.481307,1126614144148477771,1.213618e+07,2021-05-07
911,1620259200,3496.292879,0.000286,902.050180,68047172598720294,1.222406e+07,2021-05-06


# Uniswap V3 position with hedge

The document contains calculations for the liquidity, tokens amounts and fees. The formulae may be found in the paper [paper](https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf) by Atis Elsts and in the original [whitepaper](https://uniswap.org/whitepaper-v3.pdf) for Uniswap V3. 

## Load Uniswap V3 pool data using TheGraph

In [29]:
# connect to the Uniswap V3 Subgraph
uni_transport = RequestsHTTPTransport(
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3', #here we can use uniswap-v3
    verify = True,
    retries=3,  
)
client = Client(transport = uni_transport)

In [30]:
data_pools = []

In [31]:
# get the set of pools available in the Uniswap V3 Subgraph
# the examples for Uniswap are available in the Uniswap documentation: https://docs.uniswap.org/api/subgraph/guides/examples
# GraphQL tutorials: https://thegraph.com/docs/en/querying/querying-best-practices/

query = gql(
    """
query ($first: Int!, $skip: Int!) { 
  pools (first: $first, skip: $skip){
    id
    volumeUSD
    token0{
        name
    }
    token1{
        name
    }
  }
}
"""
)

page_size = 1000
skip = 0

while True:
    if skip > 5000:
        break 
    vars = {"first": page_size, "skip": skip}
    result = client.execute(query, variable_values=vars)
    
    data = []
    for i in result['pools']:
        data.append([
            i['id'],
            i['volumeUSD'],
            i['token0']['name'],
            i['token1']['name']
        ])
        
    df = pd.DataFrame(data)
    df.columns = ['id','volumeUSD','name0','name1']
    df['Pair'] = df['name0'] + '-' + df['name1']
    df['volumeUSD'] = pd.to_numeric(df['volumeUSD'])     
        
    data_pools.extend(df.to_dict('records'))
    skip += page_size
    
    if not result['pools']:
        break


data_pools = pd.DataFrame(data_pools)
data_pools

,id,volumeUSD,name0,name1,Pair
0,0x0000d36ab86d213c14d93cd5ae78615a20596505,3215.847063,Rekt Ralph,Wrapped Ether,Rekt Ralph-Wrapped Ether
1,0x0001fcbba8eb491c3ccfeddc5a5caba1a98c4c28,0.184919,BinanceCZ,Wrapped Ether,BinanceCZ-Wrapped Ether
2,0x0002e63328169d7feea121f1e32e4f620abf0352,0.000000,Wrapped NXM,ichi.farm,Wrapped NXM-ichi.farm
3,0x0003c9d75e102b8a9fe77280ca410e9135beb150,2.661070,ButtonTranche bWBTC A,Tether USD,ButtonTranche bWBTC A-Tether USD
4,0x0004cd8474e882278e32e584699090be496f410e,128211.039294,ZERO,USD Coin,ZERO-USD Coin
...,...,...,...,...,...
5995,0x58e3cb6e6d99f4814c661676b8038bd863592ab0,4800.866918,MTV,Wrapped Ether,MTV-Wrapped Ether
5996,0x58e6ec50aff3f3abb683d6bddd6d0d64cb816d0b,261458.733339,DOVU,Wrapped Ether,DOVU-Wrapped Ether
5997,0x58e8306aa212139de377f21bb99b45f8c44b679d,406504.777151,Governance OHM,USD Coin,Governance OHM-USD Coin
5998,0x58e89bbb6b0c0f447dc04dc17894bc1872cb2e74,14881.272706,Wrapped Ether,BennedCoin,Wrapped Ether-BennedCoin


In [32]:
# select the largest pools with at least $1B in trading volume

data_pools = data_pools.reset_index()
data_pools['volumeUSD'] = data_pools['volumeUSD'].astype('int64')
large_pools = data_pools[ data_pools['volumeUSD'] > 1_000_000_000]
large_pools

,index,id,volumeUSD,name0,name1,Pair
588,588,0x07f3d316630719f4fc69c152f397c150f0831071,1065164552,Euro Tether,Tether USD,Euro Tether-Tether USD
1128,1128,0x109830a1aaad605bbf02a9dfa7b0b92ec2fb7daa,3185052006,Wrapped liquid staked Ether 2.0,Wrapped Ether,Wrapped liquid staked Ether 2.0-Wrapped Ether
1189,1189,0x11950d141ecb863f01007add7d1a342041227b58,3240829585,Pepe,Wrapped Ether,Pepe-Wrapped Ether
1204,1204,0x11b815efb8f581194ae79006d24e0d814b7697f6,70669644203,Wrapped Ether,Tether USD,Wrapped Ether-Tether USD
1443,1443,0x151ccb92bc1ed5c6d0f9adb5cec4763ceb66ac7f,2749097636,ETH 2x Flexible Leverage Index,Wrapped Ether,ETH 2x Flexible Leverage Index-Wrapped Ether
1703,1703,0x18d96b617a3e5c42a2ada4bc5d1b48e223f17d0d,4133394213,USD Coin,Wrapped UST Token,USD Coin-Wrapped UST Token
1997,1997,0x1c98562a2fab5af19d8fb3291a36ac3c618835d9,1538468299,Metis Token,Wrapped Ether,Metis Token-Wrapped Ether
2043,2043,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,4697989390,Uniswap,Wrapped Ether,Uniswap-Wrapped Ether
2561,2561,0x24ee2c6b9597f035088cda8575e9d5e15a84b9df,1101004644,Quant,Wrapped Ether,Quant-Wrapped Ether
2823,2823,0x290a6a7460b308ee3f19023d2d00de604bcf5b42,4300874122,Matic Token,Wrapped Ether,Matic Token-Wrapped Ether


In [33]:
fig = px.bar(large_pools, x='Pair', y='volumeUSD')
fig.show()

## Liquidity provision in Uniswap V3

USDC - WETH pool: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640

In [34]:
data_pool = []

# query for the USDC-WETH pool
query = gql(
    """
    {
        poolDayDatas(
            first: 1000
            orderBy: date
            orderDirection: desc
            where: {pool: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"}
        ) {
            date
            token0Price
            token1Price
            feesUSD
            liquidity
        }
    }
    """
)   


result = client.execute(query, variable_values=vars)

data = []
for i in result['poolDayDatas']:
    data.append([
        i['date'],
        i['token0Price'],
        i['token1Price'],
        i['feesUSD'],
        i['liquidity']
    ])


df = pd.DataFrame(data)
df.columns = ['date', 'token0Price', 'token1Price', 'feesUSD', 'liquidity']

df['token0Price'] = pd.to_numeric(df['token0Price'])
df['token1Price'] = pd.to_numeric(df['token1Price'])
df['feesUSD'] = pd.to_numeric(df['feesUSD'])
df['USDC/WETH'] = pd.to_numeric(df['token0Price']/df['token1Price'])
df['datetime'] = pd.to_datetime(df['date']*1000000000)

df

,date,token0Price,token1Price,feesUSD,liquidity,USDC/WETH,datetime
0,1698969600,1793.186993,0.000558,47621.261999,20378604435147743781,3.215520e+06,2023-11-03
1,1698883200,1800.035082,0.000556,137531.465707,21721162802396480979,3.240126e+06,2023-11-02
2,1698796800,1846.249690,0.000542,145129.978936,20678237003634529799,3.408638e+06,2023-11-01
3,1698710400,1816.150815,0.000551,99073.055545,21509722104600588513,3.298404e+06,2023-10-31
4,1698624000,1809.635935,0.000553,115315.482355,24518778025093597202,3.274782e+06,2023-10-30
...,...,...,...,...,...,...,...
908,1620518400,3909.453752,0.000256,3165.481894,191200334840491900,1.528383e+07,2021-05-09
909,1620432000,3884.370431,0.000257,10666.832264,87209657035771435,1.508833e+07,2021-05-08
910,1620345600,3483.701459,0.000287,12066.481307,1126614144148477771,1.213618e+07,2021-05-07
911,1620259200,3496.292879,0.000286,902.050180,68047172598720294,1.222406e+07,2021-05-06


In [35]:
data_pool = df.copy()

In [36]:
data_pool['datetime'] = data_pool['datetime'].dt.date
data_pool = data_pool[['token0Price', 'token1Price', 'feesUSD', 'datetime']]
data_pool

,token0Price,token1Price,feesUSD,datetime
0,1793.186993,0.000558,47621.261999,2023-11-03
1,1800.035082,0.000556,137531.465707,2023-11-02
2,1846.249690,0.000542,145129.978936,2023-11-01
3,1816.150815,0.000551,99073.055545,2023-10-31
4,1809.635935,0.000553,115315.482355,2023-10-30
...,...,...,...,...
908,3909.453752,0.000256,3165.481894,2021-05-09
909,3884.370431,0.000257,10666.832264,2021-05-08
910,3483.701459,0.000287,12066.481307,2021-05-07
911,3496.292879,0.000286,902.050180,2021-05-06


To construct the pseudo delta neutral strategy we need to use perps. We'll take the data on ETH perps from the Binance API

In [37]:
# funding rates for ETH

url = 'https://fapi.binance.com/fapi/v1/fundingRate'
symbol = 'ETHUSDT'

start_time = dt.datetime(2021, 5, 5).timestamp() * 1000
end_time = dt.datetime(2023, 10, 26).timestamp() * 1000

data = requests.get(url, params={'symbol': symbol, 'startTime': int(start_time)}).json()
last_time = data[-1]['fundingTime']


while last_time <= end_time:
    data.extend(requests.get(url, params={'symbol': symbol, 'startTime': int(last_time)}).json())
    last_time = data[-1]['fundingTime']

df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['fundingTime'], unit='ms')
df['fundingRate'] = pd.to_numeric(df['fundingRate'])
df = df.drop_duplicates()
df

,symbol,fundingTime,fundingRate,markPrice,datetime
0,ETHUSDT,1620172800001,0.000384,,2021-05-05 00:00:00.001
1,ETHUSDT,1620201600007,0.000783,,2021-05-05 08:00:00.007
2,ETHUSDT,1620230400001,0.000880,,2021-05-05 16:00:00.001
3,ETHUSDT,1620259200003,0.000682,,2021-05-06 00:00:00.003
4,ETHUSDT,1620288000001,0.000666,,2021-05-06 08:00:00.001
...,...,...,...,...,...
2760,ETHUSDT,1698883200001,0.000268,1848.05576938,2023-11-02 00:00:00.001
2761,ETHUSDT,1698912000000,0.000206,1836.13000000,2023-11-02 08:00:00.000
2762,ETHUSDT,1698940800000,0.000100,1803.52000000,2023-11-02 16:00:00.000
2763,ETHUSDT,1698969600000,0.000100,1800.57006471,2023-11-03 00:00:00.000


In [38]:
data_funding = df.copy()

In [39]:
data_funding['datetime'] = data_funding['datetime'].dt.date
data_funding = data_funding.groupby('datetime').sum()
data_funding = data_funding.reset_index()
data_funding = data_funding[['datetime', 'fundingRate']]

In [40]:
all_data = pd.merge(data_pool, data_funding, how="left", on='datetime')
all_data

,token0Price,token1Price,feesUSD,datetime,fundingRate
0,1793.186993,0.000558,47621.261999,2023-11-03,0.000200
1,1800.035082,0.000556,137531.465707,2023-11-02,0.000573
2,1846.249690,0.000542,145129.978936,2023-11-01,0.000300
3,1816.150815,0.000551,99073.055545,2023-10-31,0.000300
4,1809.635935,0.000553,115315.482355,2023-10-30,0.000300
...,...,...,...,...,...
908,3909.453752,0.000256,3165.481894,2021-05-09,0.001547
909,3884.370431,0.000257,10666.832264,2021-05-08,0.001475
910,3483.701459,0.000287,12066.481307,2021-05-07,0.001132
911,3496.292879,0.000286,902.050180,2021-05-06,0.002458


# Startegy Backtest

Liquidity provision strategy: put capital into the Uniswap V3 pool and use perps to hedge the position. 

In [41]:
class UniswapV3Position:

    def __init__(self, token0_amount: float, token1_amount: float, liquidity: float,
                 price_current: float, price_upper: float, price_lower: float):
        self._token0_amount: float = token0_amount
        self._token1_amount: float = token1_amount
        self._liquidity: float = liquidity
        self._price_current: float = price_current
        self._price_init: float = price_current
        self._price_upper: float = price_upper
        self._price_lower: float = price_lower
        self._acc_fees: float = 0

    def update_state(self, price: float) -> None:
        if price < self._price_lower:
            self._token0_amount = 0
            self._token1_amount = self._liquidity * (1/(self._price_lower**0.5) - 1/(self._price_upper**0.5))
        elif self._price_lower <= price < self._price_upper:
            self._token0_amount = self._liquidity * (price**0.5 - self._price_lower**0.5)
            self._token1_amount = self._liquidity * (1/(price**0.5) - 1/(self._price_upper**0.5))
        else:
            self._token0_amount = self._liquidity * (self._price_upper**0.5 - self._price_lower**0.5)
            self._token1_amount = 0
        self._price_current = price

    def balance(self, side: bool = True) -> float:
        if side:
            return self._token0_amount + self._token1_amount * self._price_current + self._acc_fees
        return self._token0_amount / self._price_current + self._token1_amount + self._acc_fees

    @classmethod
    def price_to_tick(cls, price: float) -> float:
        return math.floor(math.log(price, 1.0001))

    @classmethod
    def tick_to_price(clas, tick: float) -> float:
        return 1.0001 ** tick
    
    def __str__(self) -> str:
        return f"token0: {self._token0_amount}, token1: {self._token1_amount}, liquidity: {self._liquidity}, price_current: {self._price_current}, price_upper: {self._price_upper}, price_lower: {self._price_lower}"
    
    def __repr__(self) -> str:
        return self.__str__()


def create_position_by_notional(
            deposit_amount_in_notional: float,
            price_current: float, price_upper: float, price_lower: float,
    ) -> UniswapV3Position:
        X = deposit_amount_in_notional

        liquidity = (X/2/price_current) / (1/(price_current**0.5) - 1/(price_upper**0.5))
        token0_amount = liquidity * (price_current**0.5 - price_lower**0.5)

        ratio = token0_amount / (X/2/price_current)
        new_deposit_amount = X/(ratio + price_current)

        token1_amount = new_deposit_amount
        liquidity = new_deposit_amount /  (1/(price_current**0.5) - 1/(price_upper**0.5))
        token0_amount = liquidity * (price_current**0.5 - price_lower**0.5)

        return UniswapV3Position(token0_amount, token1_amount, liquidity, price_current, price_upper, price_lower)

In [42]:
position = create_position_by_notional(100_000, 1000, 1500, 500)
position.balance()
assert position.balance() == 100_000

In [43]:
position.update_state(1600)
assert position._token1_amount == 0
assert position.balance() > 100_000

position.update_state(100)
assert position._token0_amount == 0
assert position.balance() < 100_000


In [44]:
class HedgePosition:
    def __init__(self, size: float, entry_price: float):
        self._size: float = size
        self._entry_price: float = entry_price
        self._current_price: float = entry_price

    def update_state(self, price: float) -> None:
        self._current_price = price

    def pnl(self) -> float:
        return self._size * (self._current_price - self._entry_price)
    
    def __str__(self) -> str:
        return f'BaseHedgePosition(size={self._size}, entry_price={self._entry_price}, current_price={self._current_price})'


class Hedge:
    def __init__(self):
        self._position: HedgePosition = None
        self._balance: float = 0
        self._current_price: float = None
        self._trading_fee: float = 0.01

    def deposit(self, amount: float):
        if amount <= 0:
            raise Exception(f'Cannot deposit non-positive amount {amount}')
        self._balance += amount

    def withdraw(self, amount: float):
        if amount > self._balance:
            raise Exception(f'Not enough balance to withdraw {amount}, available {self._balance}')
        self._balance -= amount

    @property
    def balance(self) -> float:
        return self._balance

    @property
    def position(self) -> HedgePosition:
        return self._position

    def margin_balance(self) -> float:
        if not self._position:
            return self._balance
        return self._balance + self._position.pnl()

    def update_state(self, price: float, funding: float) -> None:
        self._current_price = price
        if self._position:
            self._position.update_state(price)
            self._balance += funding * abs(self._position._size)
        self.__check_liquidation()

    def open_position(self, size: float, entry_price: float) -> None:
        if self._position:
            raise Exception(f'Cannot open position, already have one {self._position}')
        
        if size > 0:
            raise Exception(f'Cannot open short position {size}')

        self._position = HedgePosition(size, entry_price)
        self._balance -= abs(size) * entry_price * self._trading_fee

    def close_position(self) -> None:
        if not self._position:
            return
        # self._balance -= abs(self._position._size) * self._current_price * self._trading_fee
        self._balance += self._position.pnl()
        self._position = None

    def __check_liquidation(self):
        if self._position:
            # liquidate if margin balance is 5% of the position size
            if self.margin_balance() < abs(self._position._size) * 0.05:
                self._balance = 0
                self._position = None

In [45]:
hedge = Hedge()
hedge.deposit(100_000)
assert hedge.balance == 100_000
assert hedge.margin_balance() == 100_000
hedge.update_state(1000, -1)
assert hedge.margin_balance() == 100_000


In [46]:
hedge.open_position(
    size=-100,
    entry_price=1000
)
assert hedge.balance == 100_000 - 100_000 * 0.01
assert hedge.margin_balance() == 100_000 - 100_000 * 0.01

In [47]:
hedge.position._size * 1000 / hedge.margin_balance()

-1.0101010101010102

In [48]:
hedge.update_state(1500, -0.01)
hedge.margin_balance()

48999.0

In [49]:
class Strategy:

    params: dict = {
        'STD_COUNT': 3,
        'START_EQUITY': 100_000,
        'STD_TIMERANGE': 30, # days
        'POOL_APY': 0.40
    }

    def __init__(self, 
                 states: pd.DataFrame,
                ):
        self._hedge: Hedge = Hedge()
        self._pool_position: UniswapV3Position = None
        self._states: pd.DataFrame = states
        self._data = []

    def calculate_std(self, timestamp: dt.datetime) -> float:
        return abs(self._states[self._states['datetime'] <= timestamp]['token0Price'].pct_change().std())

    def run(self):

        for i, state in tqdm(self._states.iterrows()):
            if i < self.params['STD_TIMERANGE']:
                print(f'Strategy | Skipping {state["datetime"]}')
                continue

            self._hedge.update_state(state['token0Price'], state['fundingRate'])

            if self._pool_position:
                self._pool_position.update_state(state['token0Price'])
                self._pool_position._acc_fees += self._pool_position.balance() * self.params['POOL_APY'] / 365 # daily fees, TODO: add exact approximation https://www.metacrypt.org/tools
                if self._pool_position._token0_amount * self._pool_position._token1_amount == 0:
                    self.rebalance(state)
            else:
                self.rebalance(state)

            self._data.append([
                state['datetime'],
                state['token0Price'],
                self._pool_position.balance(),
                self._hedge.margin_balance(),
                self._pool_position.balance() + self._hedge.balance,
                self._pool_position._price_upper,
                self._pool_position._price_lower,
            ])

        return pd.DataFrame(self._data, columns=['datetime', 'price', 'pool_balance', 'hedge_balance', 'total_balance', 'price_upper', 'price_lower'])

    def rebalance(self, state):
        
        print(f'Strategy | Start rebalancing at {state["datetime"]}')

        std = self.calculate_std(state['datetime'])
        print(f'Strategy | Calculated std {std}')

        if self._pool_position is not None:
            pool_balance: float = self._pool_position.balance() * (1 - 0.0005) # some fees and slippage
            self._hedge.close_position()
            equity: float = pool_balance + self._hedge.margin_balance()
        else:
            equity: float = self.params['START_EQUITY']

        print(f'Strategy | Start balances: pool {self._pool_position.balance() if self._pool_position else 0}, hedge {self._hedge.margin_balance()}, equity {equity}')

        # virtual witfhdraw and deposit for rebalancing capital
        self._hedge.withdraw(self._hedge.margin_balance())
        self._hedge.deposit((1/3) * equity)

        self._pool_position = create_position_by_notional(
            (2/3)*equity,
            price_current=state['token0Price'],
            price_upper=state['token0Price'] * (1 + std * self.params['STD_COUNT']),
            price_lower=state['token0Price'] * (1 - std * self.params['STD_COUNT']),
        )
        self._hedge.open_position(
            size=-(self._pool_position.balance() / 2) / state['token0Price'],
            entry_price=state['token0Price'],
        )
        print(f'Strategy | Created pool position {self._pool_position}')
        print(f'Strategy | New balances: pool {self._pool_position.balance()}, hedge {self._hedge.margin_balance()}, equity {self._pool_position.balance() + self._hedge.margin_balance()}')
        print(f'Strategy | End rebalancing at {state["datetime"]}')



In [50]:
# you can take filtered data
# filtered_data = all_data[all_data['datetime'] > dt.datetime(2021, 5, 5).date()]
strategy = Strategy(all_data.sort_values('datetime').reset_index())
data = strategy.run()

282it [00:00, 2811.59it/s]

Strategy | Skipping 2021-05-05
Strategy | Skipping 2021-05-06
Strategy | Skipping 2021-05-07
Strategy | Skipping 2021-05-08
Strategy | Skipping 2021-05-09
Strategy | Skipping 2021-05-10
Strategy | Skipping 2021-05-11
Strategy | Skipping 2021-05-12
Strategy | Skipping 2021-05-13
Strategy | Skipping 2021-05-14
Strategy | Skipping 2021-05-15
Strategy | Skipping 2021-05-16
Strategy | Skipping 2021-05-17
Strategy | Skipping 2021-05-18
Strategy | Skipping 2021-05-19
Strategy | Skipping 2021-05-20
Strategy | Skipping 2021-05-21
Strategy | Skipping 2021-05-22
Strategy | Skipping 2021-05-23
Strategy | Skipping 2021-05-24
Strategy | Skipping 2021-05-25
Strategy | Skipping 2021-05-26
Strategy | Skipping 2021-05-27
Strategy | Skipping 2021-05-28
Strategy | Skipping 2021-05-29
Strategy | Skipping 2021-05-30
Strategy | Skipping 2021-05-31
Strategy | Skipping 2021-06-01
Strategy | Skipping 2021-06-02
Strategy | Skipping 2021-06-03
Strategy | Start rebalancing at 2021-06-04
Strategy | Calculated std 0

913it [00:00, 2561.87it/s]

Strategy | Calculated std 0.05028655112454224
Strategy | Start balances: pool 26542.047561788215, hedge 16998.018435008606, equity 43526.79497301593
Strategy | Created pool position token0: 15566.5379233431, token1: 10.791318190858396, liquidity: 5615.7962303092945, price_current: 1246.4951133954903, price_upper: 1434.5409341342536, price_lower: 1058.449292656727
Strategy | New balances: pool 29017.86331534395, hedge 14363.842341095256, equity 43381.70565643921
Strategy | End rebalancing at 2022-09-21
Strategy | Start rebalancing at 2022-10-25
Strategy | Calculated std 0.049080727861537124
Strategy | Start balances: pool 31120.493437846482, hedge 11875.293068916799, equity 42980.226260044365
Strategy | Created pool position token0: 15346.778942273935, token1: 9.112348384703317, liquidity: 5246.208223573633, price_current: 1460.2937321214172, price_upper: 1675.3105699138962, price_lower: 1245.2768943289382
Strategy | New balances: pool 28653.484173362907, hedge 14183.47466581464, equity

In [51]:
df = pd.DataFrame(data)
df

,datetime,price,pool_balance,hedge_balance,total_balance,price_upper,price_lower
0,2021-06-04,2689.464730,66666.666667,33000.000000,99666.666667,3456.876255,1922.053205
1,2021-06-05,2617.738662,65925.373321,33888.979473,98925.377039,3456.876255,1922.053205
2,2021-06-06,2704.825177,66974.929163,32809.629444,99974.936600,3456.876255,1922.053205
3,2021-06-07,2578.338525,65585.328119,34377.313779,98585.339273,3456.876255,1922.053205
4,2021-06-08,2503.871310,64662.256826,35300.267136,97662.271699,3456.876255,1922.053205
...,...,...,...,...,...,...,...
878,2023-10-30,1809.635935,27833.743359,13191.304870,41365.376658,1988.913269,1542.431285
879,2023-10-31,1816.150815,27900.884558,13140.874651,41432.520179,1988.913269,1542.431285
880,2023-11-01,1846.249690,28082.026145,12907.877652,41613.664088,1988.913269,1542.431285
881,2023-11-02,1800.035082,27868.944090,13265.635411,41400.586472,1988.913269,1542.431285


In [52]:
# scale all balances to first day
df['pool_balance'] = df['pool_balance'] / df['pool_balance'][0]
df['hedge_balance'] = df['hedge_balance'] / df['hedge_balance'][0]
df['total_balance'] = df['total_balance'] / df['total_balance'][0]

In [53]:
px.line(df.sort_values('datetime'), x='datetime', y=['pool_balance', 'hedge_balance', 'total_balance'])

In [54]:
px.line(df, x='datetime', y=['price', 'price_upper', 'price_lower'])